# Advanced Indexing


## Learning Objectives

* Vectorized Indexing 
* Dropping/Masking Data Using `where` and `isin`
* Fancy DateTime Indexing


## Overview


In the pervious notebooks, we learned basic forms of indexing with xarray (positional and name based dimensions, integer and label based indexing) and nearest neighbor lookups. In this tutorial, we will discover more advanced options for vectorized indexing and learn about additional useful methods relating to indexing/selecting data such as masking. 


First, let's import packages: 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In this tutorial, we’ll use air temperature tutorial dataset from the National Center for Environmental Prediction.  

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
da = ds.air
ds

## Vectorized Indexing

Like NumPy and pandas, Xarray supports indexing many array elements at once in a
*vectorized* manner. 

If you only provide integers, slices, or unlabeled arrays (array without dimension names, such as `np.ndarray`, `list`, but not `DataArray()`) indexing can be understood as orthogonally.

In [ ]:
da[0,[2,4,10,13],[1,6,7]].plot();

But for more flexibility, you can supply `DataArray()` objects as indexers. 

Vectorized indexing using `DataArrays()` may be used to extract information from the nearest grid cells of interest, for example, the nearest climate model grid cells to a collection specified weather station latitudes and longitudes.

**To trigger vectorized indexing behavior you will need to provide the selection dimensions with a new shared output dimension name.** 

In the example below, the selections of the closest latitude and longitude are renamed to an output dimension named `points`:

In [ ]:
# Define target latitude and longitude (where weather stations might be)
lat_points = xr.DataArray([31, 41, 42, 42], dims="points")
lon_points = xr.DataArray([200, 201, 202, 205], dims="points")
lat_points

In [ ]:
lon_points

Now, retrieve data at the grid cells nearest to the target latitudes and longitudes (weather stations):

In [ ]:
da.sel(lat=lat_points, lon=lon_points,method="nearest").plot();

As we can see in the above example, the dimensions on resultant arrays are given by the ordered union of the indexers’ dimensions.

In [ ]:
da.sel(lat=lat_points, lon=lon_points,method="nearest").dims

Please note that slices or sequences/arrays without named-dimensions are treated as if they have the same dimension which is indexed along:

In [ ]:
da.sel(lat=[20, 30, 40], lon=lon_points,method="nearest")

<div class="alert alert-block alert-warning">
    
<strong>Warning:</strong> If an indexer is a DataArray(), its coordinates should not conflict with the selected subpart of the target array (except for the explicitly indexed dimensions with .loc/.sel). Otherwise, IndexError will be raised.
    
</div>


## Masking with `where()`

Indexing methods on Xarray objects generally return a subset of the original data. However, it is sometimes useful to select an object with the same shape as the original data, but with some elements masked. To do this type of selection in Xarray, use `where()`:

In [ ]:
# Let's replace the missing values (nan) with some placeholder

ds.air.where(ds.air.notnull(), -9999)

We can also select a condition to create a mask. For example, here we want to mask all the points with latitudes above 60 N. 

In [ ]:
da[0,:,:].plot();

In [ ]:
da_masked = da.where(da.lat<60)
da_masked[0,:,:].plot();

By default where maintains the original size of the data. You can use of the option `drop=True` to clips coordinate elements that are fully masked:


In [ ]:
da_masked = da.where(da.lat<60, drop=True)
da_masked[0,:,:].plot();

## Selecting Values with `isin`

To check whether elements of an xarray object contain a single object, you can compare with the equality operator `==` (e.g., `arr == 3`). To check multiple values, use `isin()`:



Here is a simple example: 

In [ ]:
x_da = xr.DataArray([1, 2, 3, 4, 5], dims=["x"])

#-- select points with values equal to 2 and 4:
x_da.isin([2, 4])

`isin()` works particularly well with `where()` to support indexing by arrays that are not already labels of an array. 

For example, we have another DataArray that displays the status flags of the data-collecting device for our data. Here, flags with value 0 and -1 signifies the device was functioning correctly, while 0 indicates a malfunction, implying that the resulting data collected may not be accurate.

In [ ]:
flags = xr.DataArray(
    np.random.randint(-1, 5, da.shape),
    dims=da.dims,
    coords=da.coords
)
flags

Now, we want to only see the data for points where out measurement device is working correctly: 

In [ ]:
da_masked = da.where(flags.isin([1,2,3,4,5]), drop=True)
da_masked[0,:,:].plot();

<div class="alert alert-block alert-warning">
    
<strong>Warning:</strong> Please note that when done repeatedly, this type of indexing is significantly slower than using sel().
    
</div>

## Align and Reindex 

Xarray enforces alignment between index Coordinates (that is, coordinates with the same name as a dimension, marked by *) on objects used in binary operations.


In [ ]:
da

In [240]:
da = xr.DataArray(
    np.random.rand(4, 3),
    [
        ("time", pd.date_range("2000-01-01", periods=4)),
        ("space", ["IL", "IA", "IN"]),
    ],
)
da

<xarray.DataArray (time: 4, space: 3)>
array([[0.41063639, 0.64438845, 0.34046896],
       [0.87939503, 0.1581048 , 0.34112795],
       [0.85912828, 0.92362903, 0.77612158],
       [0.76501244, 0.20228242, 0.94031457]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IL' 'IA' 'IN'

In [241]:
da.reindex(space=["IA", "CA"])

<xarray.DataArray (time: 4, space: 2)>
array([[0.64438845,        nan],
       [0.1581048 ,        nan],
       [0.92362903,        nan],
       [0.20228242,        nan]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'IA' 'CA'

In [236]:
new_time = pd.date_range('2013-02-01', periods=20, freq='H')

In [237]:
da.reindex(time=new_time)

<xarray.DataArray 'air' (time: 20, lat: 25, lon: 53)>
array([[[238.09999, 238.29999, 238.2    , ..., 238.7    , 240.09999,
         241.7    ],
        [241.2    , 240.5    , 239.2    , ..., 234.89   , 236.     ,
         238.39   ],
        [244.29999, 244.     , 243.09999, ..., 235.59999, 237.79999,
         241.59999],
        ...,
        [296.     , 295.5    , 295.4    , ..., 294.6    , 295.19998,
         295.4    ],
        [297.1    , 297.     , 296.69998, ..., 295.9    , 296.5    ,
         296.1    ],
        [298.1    , 298.     , 297.4    , ..., 296.79   , 297.6    ,
         297.29   ]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
...
        [294.6    , 293.6    , 293.19998, ..., 295.79   , 296.29   ,
         296.4    ],
        [295.9    , 295.4    , 294.69998, ..., 297.1    , 297.5    ,
         296.9    ],
        [297.1    , 296.69998, 296.     , ..., 297.69998, 297.79   ,
         297.     ]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-02-01 ... 2013-02-01T19:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

## Fancy DateTime Indexing 


## Additional Resources

- [Xarray Docs - Indexing and Selecting Data](https://docs.xarray.dev/en/stable/indexing.html)
